In [1]:
from Bio.Blast import NCBIXML 
from Bio.Blast import NCBIWWW
import requests, sys, json
import re
from Bio import SeqIO
from Bio import Entrez
from Bio import Medline
import pandas as pd
import numpy as np
from tqdm import tqdm
from Bio import SearchIO
from Bio.SwissProt import KeyWList
from Bio import SwissProt
from Bio import ExPASy
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.Seq import Seq
from Bio import AlignIO
from Bio.Align.Applications import ClustalwCommandline
from Bio.Align import AlignInfo
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.Phylo.TreeConstruction import DistanceCalculator
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
from Bio import Phylo

# Literature Analysis

## Automated searches with Biopython

Two possible strategies for searching for articles.
The first allows searching globally, the second is more targeted and involves a pre-selection of the articles.

In [2]:
#First Strategie

database = "PubMed"#For literature will be PubMed
word = "DDX18"
res= "5" #I choose a limit of thirty
email= "guilherme.lobo@ua.pt"

Entrez.email= email
handle=Entrez.esearch(db = database, term=word, retmax= res)
record=Entrez.read(handle)
handle.close()
idlist= record['IdList']

handle = Entrez.efetch(db=database, id=idlist, rettype="medline", retmode="text") #para este caso em especifico devemos ter como database a PubMed("Medline")
records = Medline.parse(handle)

for record in records:
    #print(record) 
    #print("PMID:",record.get("PMID","-"))
    print("title:", record.get("TI", "-"))
    #print('abstract:', record.get('AB', '-'))
    print("authors:", record.get("AU", "-"))
    print("source:", record.get("SO", "-"))
    print("")

title: Detection of Selection Signatures in Anqing Six-End-White Pigs Based on Resequencing Data.
authors: ['Chen Y', 'Wu X', 'Wang J', 'Hou Y', 'Liu Y', 'Wang B', 'Hu X', 'Zheng X', 'Zhang X', 'Ding Y', 'Yin Z']
source: Genes (Basel). 2022 Dec 8;13(12):2310. doi: 10.3390/genes13122310.

title: DDX18 prevents R-loop-induced DNA damage and genome instability via PARP-1.
authors: ['Lin WL', 'Chen JK', 'Wen X', 'He W', 'Zarceno GA', 'Chen Y', 'Chen S', 'Paull TT', 'Liu HW']
source: Cell Rep. 2022 Jul 19;40(3):111089. doi: 10.1016/j.celrep.2022.111089.

title: Identification of Important Modules and Hub Gene in Chronic Kidney Disease Based on WGCNA.
authors: ['Wang J', 'Yin Y', 'Lu Q', 'Zhao YR', 'Hu YJ', 'Hu YZ', 'Wang ZY']
source: J Immunol Res. 2022 May 4;2022:4615292. doi: 10.1155/2022/4615292. eCollection 2022.

title: The RNA-Binding Protein DDX18 Promotes Gastric Cancer by Affecting the Maturation of MicroRNA-21.
authors: ['Zhang Y', 'Yu F', 'Ni B', 'Li Q', 'Bae SW', 'Choi JH', 'Yan

In [3]:
#Same strategie but for type II diabetes

database = "PubMed"
word = "Type 2 Diabetes"
res= "5" 
email= "guilherme.lobo@ua.pt"

Entrez.email= email
handle=Entrez.esearch(db = database, term=word, retmax= res)
record=Entrez.read(handle)
handle.close()
idlist= record['IdList']

handle = Entrez.efetch(db=database, id=idlist, rettype="medline", retmode="text")
records = Medline.parse(handle)

for record in records:
    print("title:", record.get("TI", "-"))
    print("authors:", record.get("AU", "-"))
    print("source:", record.get("SO", "-"))
    print("")

title: Association of metformin exposure with low risks of frailty and adverse outcomes in patients with diabetes.
authors: ['Liu P', 'Pan Y', 'Song Y', 'Zhou Y', 'Zhang W', 'Li X', 'Li J', 'Li Y', 'Ma L']
source: Eur J Med Res. 2023 Feb 3;28(1):65. doi: 10.1186/s40001-023-01017-6.

title: Dietary habits after a physical activity mHealth intervention: a randomized controlled trial.
authors: ['Sjoblom L', 'Bonn SE', 'Alexandrou C', 'Dahlgren A', 'Eke H', 'Trolle Lagerros Y']
source: BMC Nutr. 2023 Feb 2;9(1):23. doi: 10.1186/s40795-023-00682-4.

title: Association of triglycerides to high-density lipoprotein cholesterol ratio to identify future prediabetes and type 2 diabetes mellitus: over one-decade follow-up in the Iranian population.
authors: ['Tohidi M', 'Asgari S', 'Chary A', 'Safiee S', 'Azizi F', 'Hadaegh F']
source: Diabetol Metab Syndr. 2023 Feb 2;15(1):13. doi: 10.1186/s13098-023-00988-0.

title: Association of BMAL1 clock gene polymorphisms with fasting glucose in children.


In [3]:
#Second Strategie for myarticles_DDX18; articles used in literature analysis
alldata=[]
with open("Article_DDX18.txt",encoding="utf-8") as f: #For this examples I chosse articles from PubMed about Diabetes type II
    pmids= Medline.parse(f)
    for pmid in tqdm(pmids):
        try:
            pid=pmid["PMID"]
        except:
            pid="-"
        try:
            Title=pmid["TI"]
        except:
            Title="-"
        #try:        
            #Abstract=pmid["AB"]
        #except:
            #Abstract="-"
        #try:
            #Authors=pmid["AU"]
        #except:
            #Authors="-"
        #try:
            #Source=pmid["SO"]
        #except:
            #Source="-"
        
        dic={"PMID":pid,
             "Title":Title,
             #"Abstract":Abstract,
             #"Authors":Authors,
             #"Source":Source
            }
        alldata.append(dic)

df = pd.DataFrame.from_records(alldata,index=['1', '2',"3","4","5"])
print(df)
#writer = pd.ExcelWriter('articles_ddx18.xlsx')
#df.to_Exel(writer)
#writer.save()
#df=pd.DataFrame(alldata)
#df.to_csv("articles_ddx18.csv", index=False)

5it [00:00, 2717.93it/s]

       PMID                                              Title
1  33489896  The RNA-Binding Protein DDX18 Promotes Gastric...
2  18351129                  Cellular studies of MrDb (DDX18).
3  34603195  Non-Coding RNA as Biomarkers for Type 2 Diabet...
4  33479058  The First Genome-Wide Association Study for Ty...
5  30297969  Fine-mapping type 2 diabetes loci to single-va...


# Analysis of the sequence and features present in the NCBI

## DDX18

In [4]:
database ="nucleotide"
word = "ddx18 and homo sapiens and Chromosome 2 and not predicted and not unverified"
res= "30"
Entrez.email="guilherme.lobo@ua.pt"
handle=Entrez.esearch(db = database, term=word, retmax= res)
record=Entrez.read(handle)
handle.close()
idlist= record['IdList']

handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for x in records:
    print(x.id,x.description)

NM_006773.4 Homo sapiens DEAD-box helicase 18 (DDX18), mRNA
AC009312.4 Homo sapiens BAC clone RP11-425F6 from 2, complete sequence
AC009404.5 Homo sapiens BAC clone RP11-28H22 from 2, complete sequence
CM000253.1 Homo sapiens chromosome 2, whole genome shotgun sequence
CH471103.1 Homo sapiens 211000035839809 genomic scaffold, whole genome shotgun sequence
DQ655976.2 Homo sapiens clone Affy08254D01, mRNA sequence
DQ655975.2 Homo sapiens clone Affy08248B12, mRNA sequence
NT_086328.2 Homo sapiens chromosome 2 sequence, ENCODE region ENr121


Taking into account the description given, we think that the id (**"NM_006773.4"**) would be the most appropriate and, as such, will be used from now on.

In [5]:
with open("nucleotide_ddx18.fasta","w") as f: 
    for rec in records:
        #print(rec)
        if rec.id == "NM_006773.4":
            print("Record sequence length: ", len(rec.seq))
            print("Record features count: ", len(rec.features))
            sequence_nucleotide_DDX18=rec.seq
            print( "Nuclleotide sequence:", str(rec.seq)) #useful for blastn
            f.write(f">{rec.id}\n{sequence_nucleotide_DDX18}\n\n") # only the sequence in the file
        
            cds = []
            for i,feature in enumerate(rec.features):
                if feature.type=="CDS":
                    cds.append(i)
                    for k in cds:
                        print("ddx18 gene synonyms:",rec.features[k].qualifiers["gene_synonym"])#can help latter in the search
                        print("name:",rec.features[k].qualifiers["product"])
                        protein_ID=rec.features[k].qualifiers["protein_id"]
                        sequence_protein=rec.features[k].qualifiers["translation"]#get the protein seq of my cds
                        print("Protein sequence:",sequence_protein)
            gene = []
            for j in range(len(rec.features)):
                if rec.features[j].type == "gene":
                    gene.append(j)
                    for g in gene:
                        print("Location:",rec.features[g].location)
                        print("Strand:",rec.features[g].location.strand)
                        print("Gene:",rec.features[g].qualifiers["gene"])
#the real important thing is the nucleotide sequence  and the protein sequence of cds

Record sequence length:  3753
Record features count:  27
Nuclleotide sequence: ACGTGCGGCCGGAAGGGAAGTAACGTCAGCCTGAGAACTGAGTAGCTGTACTGTGTGGCGCCTTATTCTAGGCACTTGTTGGGCAGAATGTCACACCTGCCGATGAAACTCCTGCGTAAGAAGATCGAGAAGCGGAACCTCAAATTGCGGCAGCGGAACCTAAAGTTTCAGGGGGCCTCAAATCTGACCCTATCGGAAACTCAAAATGGAGATGTATCTGAAGAAACAATGGGAAGTAGAAAGGTTAAAAAATCAAAACAAAAGCCCATGAATGTGGGCTTATCAGAAACTCAAAATGGAGGCATGTCTCAAGAAGCAGTGGGAAATATAAAAGTTACAAAGTCTCCCCAGAAATCCACTGTATTAACCAATGGAGAAGCAGCAATGCAGTCTTCCAATTCAGAATCAAAAAAGAAAAAGAAGAAAAAGAGAAAAATGGTGAATGATGCTGAGCCTGATACGAAAAAAGCAAAAACTGAAAACAAAGGGAAATCTGAAGAAGAAAGTGCCGAGACTACTAAAGAAACAGAAAATAATGTGGAGAAGCCAGATAATGATGAAGATGAGAGTGAGGTGCCCAGTCTGCCCCTGGGACTGACAGGAGCTTTTGAGGATACTTCGTTTGCTTCTCTATGTAATCTTGTCAATGAAAACACTCTGAAGGCAATAAAAGAAATGGGTTTTACAAACATGACTGAAATTCAGCATAAAAGTATCAGACCACTTCTGGAAGGCAGGGATCTTCTAGCAGCTGCAAAAACAGGCAGTGGTAAAACCCTGGCTTTTCTCATCCCTGCAGTTGAACTCATTGTTAAGTTAAGGTTCATGCCCAGGAATGGAACAGGAGTCCTTATTCTCTCACCTACTAGAGAACTAGCCATGCAAACCTTTGGTGTTCTTAAGGAGCTGATGACTCACCAC

In [92]:
#The correspondent information of NM_006773.4 was downloaded to a file
import os
Entrez.email = "guilherme.lobo@ua.pt"
filename = "NM_006773.4.gb"
if not os.path.isfile(filename):
    net_handle = Entrez.efetch(db="nucleotide", id="NM_006773.4", rettype="gb", retmode="text")
    out_handle = open(filename, "w")
    out_handle.write(net_handle.read())
    out_handle.close()
    net_handle.close()

In [6]:
record = SeqIO.read(open("NM_006773.4.gb"), format="genbank")
position=0
record_types={}
for x in record.features:
    record_types[x.type]=record_types.get(x.type,0)+1
for i in range(len(record.features)):
    if record.features[i].type == "CDS":
        position=i
print("The length of the sequence: {}\n".format(len(record.seq)))
print("Type of features: {}\n".format(record_types))
#print("Comment from NCBI: {}\n".format(record.annotations["comment"]))
print("Location of the CDS on the original sequence: {}".format(record.features[position].location))

The length of the sequence: 3753

Type of features: {'source': 1, 'gene': 1, 'exon': 14, 'misc_feature': 4, 'CDS': 1, 'regulatory': 3, 'polyA_site': 3}

Location of the CDS on the original sequence: [87:2100](+)


In [7]:
CDS_nuc_seq=record.seq[int(record.features[position].location.start):int(record.features[position].location.end)]

In [8]:
#saving the CDS_nucleotides and CD_aminoacid seqs in files
filename = "NM_006773.4.gb"
filename_CDS_nucl = "CDS_nucleot_DDX18_seq.fasta"
filename_CDS_prot = "CDS_prot_DDX18_seq.fasta"
input_handle  = open(filename, "r")
output_handle_nucl = open(filename_CDS_nucl, "w")
output_handle_prot = open(filename_CDS_prot, "w")
for seq_record in SeqIO.parse(input_handle, "genbank") :
    output_handle_nucl.write(">\n%s" % (CDS_nuc_seq))
    output_handle_prot.write(">\n%s" % ("".join(record.features[position].qualifiers['translation']))) #without join, output is a list
    
output_handle_prot.close()
output_handle_nucl.close()
input_handle.close()

# Homology analysis by BLAST or Diamond

## Blastn

In [9]:
record = SeqIO.read(open("nucleotide_ddx18.fasta"), format="fasta") 
print(len(record.seq))

3753


In [10]:
result_handle=NCBIWWW.qblast("blastn","nt",record.seq)
with open("blastn_DNA_ddx18_file.xml","w") as out_handle: 
    out_handle.write(result_handle.read())
result_handle.close()

In [11]:
Blast=open("blastn_DNA_ddx18_file.xml")
Blast_record=NCBIXML.parse(Blast)
for x in Blast_record:
    print(x.alignments[0])#best-one

gi|1519243738|ref|NM_006773.4| Homo sapiens DEAD-box helicase 18 (DDX18), mRNA
           Length = 3753



In [13]:
result_seq = open("blastn_DNA_ddx18_file.xml")
blast_results = NCBIXML.read(result_seq)
#print("Number of alignments:",len(blast_results.alignments))
for parameter in blast_results.alignments[0:5]:
    print()
    print('Accession: ', parameter.accession)
    print('Definition: ', parameter.hit_def)    
#     for e_v in parameter.hsps:
#         print('E-value: ', e_v.expect)


Accession:  NM_006773
Definition:  Homo sapiens DEAD-box helicase 18 (DDX18), mRNA
E-value:  0.0
E-value:  0.0225354
E-value:  0.0225354

Accession:  BC024739
Definition:  Homo sapiens DEAD (Asp-Glu-Ala-Asp) box polypeptide 18, mRNA (cDNA clone MGC:29902 IMAGE:4995871), complete cds
E-value:  0.0
E-value:  0.0225354
E-value:  0.0225354

Accession:  XM_003819127
Definition:  PREDICTED: Pan paniscus DEAD-box helicase 18 (DDX18), mRNA
E-value:  0.0
E-value:  0.000529982
E-value:  0.0225354

Accession:  XM_515753
Definition:  PREDICTED: Pan troglodytes DEAD-box helicase 18 (DDX18), mRNA
E-value:  0.0
E-value:  0.0225354
E-value:  0.0225354

Accession:  XM_004031662
Definition:  PREDICTED: Gorilla gorilla gorilla DEAD-box helicase 18 (DDX18), mRNA
E-value:  0.0
E-value:  1.02311e-06
E-value:  0.000529982


In [15]:
E_VALUE_THRESH = 0.00001
results_Blast= open('blastn_DNA_ddx18_file.xml')
blast_records = NCBIXML.read(results_Blast)
for alignment in  blast_records.alignments:
    print (alignment)
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            print('****Alignment****')
            print('sequence: ', alignment.title)
            print('lenght:', alignment.length)
            print(hsp.query[0:75] + '...')
            print(hsp.match[0:75] + '...')
            print(hsp.sbjct[0:75] + '...')
            print()
            

gi|1519243738|ref|NM_006773.4| Homo sapiens DEAD-box helicase 18 (DDX18), mRNA
           Length = 3753

****Alignment****
sequence:  gi|1519243738|ref|NM_006773.4| Homo sapiens DEAD-box helicase 18 (DDX18), mRNA
lenght: 3753
ACGTGCGGCCGGAAGGGAAGTAACGTCAGCCTGAGAACTGAGTAGCTGTACTGTGTGGCGCCTTATTCTAGGCAC...
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||...
ACGTGCGGCCGGAAGGGAAGTAACGTCAGCCTGAGAACTGAGTAGCTGTACTGTGTGGCGCCTTATTCTAGGCAC...

gi|19353238|gb|BC024739.1| Homo sapiens DEAD (Asp-Glu-Ala-Asp) box polypeptide 18, mRNA (cDNA clone MGC:29902 IMAGE:4995871), complete cds
           Length = 3764

****Alignment****
sequence:  gi|19353238|gb|BC024739.1| Homo sapiens DEAD (Asp-Glu-Ala-Asp) box polypeptide 18, mRNA (cDNA clone MGC:29902 IMAGE:4995871), complete cds
lenght: 3764
GGGAAGTAACGTCAGCCTGAGAACTGAGTAGCTGTACTGTGTGGCGCCTTATTCTAGGCACTTGTTGGGCAGAAT...
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||...
GGGAAGTAACGTCAGCCTGAGAACTGAGTAGC

In [16]:
#filtering the "predicted" alignments
results_Blastn= open('blastn_DNA_ddx18_file.xml')
blastn_records = NCBIXML.read(results_Blastn)
E_VALUE_THRESH = 0.00001
count_preditc, count_homo=0,0
list_filtered_alignments=[]
for alignment in  blastn_records.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            exist = re.search(r'PREDICTED:\s', alignment.title)
            if exist:
                pre = re.match(r'PREDICTED:\s', exist[0] )
                if pre:
                    count_preditc+=1
            else:
                list_filtered_alignments.append(alignment.accession)
                homo=re.search(r'Homo\ssapiens',alignment.title)
                if homo:
                    count_homo+=1
print(list_filtered_alignments)
print()
print('Total {} PREDICTED seqs found and remaining {} ids are from Homo sapiens'.format(count_preditc, count_homo))

['NM_006773', 'BC024739', 'NM_001132808', 'AK091227', 'AK001467', 'BC003360', 'BC001238', 'X98743', 'X98743', 'NG_008704', 'NG_008704', 'NG_008704', 'AL365434', 'AL365434', 'AL365434', 'LT744377', 'KJ897930', 'AB209392', 'AB209392', 'AB209392', 'CP034492', 'NG_002480', 'AL391262', 'NG_008706', 'AL138725']

Total 43 PREDICTED seqs found and remaining 16 ids are from Homo sapiens


In [17]:
#ids after filtration
with open('DDX18_nucleotide_result_blast.txt', 'w') as f:
    for line in list_filtered_alignments:
        f.write(f"{line}\n")

# Blastp

## DDX18

In [18]:
record_protein = SeqIO.read(open("CDS_prot_DDX18_seq.fasta"), format="fasta") 
print(len(record_protein.seq))

670


In [19]:
result_handle=NCBIWWW.qblast("blastp","swissprot",record_protein.seq)#blast with the protein translate of my cds
with open("blastp_file_ddx18.xml","w") as out_handle:
    out_handle.write(result_handle.read())
result_handle.close()

In [20]:
result_seq = open("blastp_file_ddx18.xml")
blast_results = NCBIXML.read(result_seq)
#print("Number of alignments:",len(blast_results.alignments))
for parameter in blast_results.alignments[0:5]:
    print()
    print('Accession: ', parameter.accession)
    print('Definition: ', parameter.hit_def)    
    for e_v in parameter.hsps:
        print('E-value: ', e_v.expect)


Accession:  Q9NVP1
Definition:  RecName: Full=ATP-dependent RNA helicase DDX18; AltName: Full=DEAD box protein 18; AltName: Full=Myc-regulated DEAD box protein; Short=MrDb [Homo sapiens]
E-value:  0.0

Accession:  Q8K363
Definition:  RecName: Full=ATP-dependent RNA helicase DDX18; AltName: Full=DEAD box protein 18 [Mus musculus]
E-value:  0.0

Accession:  Q9VD51
Definition:  RecName: Full=Probable ATP-dependent RNA helicase pitchoune [Drosophila melanogaster]
E-value:  0.0

Accession:  Q1EA54
Definition:  RecName: Full=ATP-dependent RNA helicase HAS1 [Coccidioides immitis RS]
E-value:  0.0

Accession:  A4R8B5
Definition:  RecName: Full=ATP-dependent RNA helicase HAS1 [Pyricularia oryzae 70-15]
E-value:  0.0


In [22]:
# results_Blastp= open('blastp_file_ddx18.xml')
# blastp_records = NCBIXML.read(results_Blastp)
# E_VALUE_THRESH = 0.001
# list_filtered_alignments,list_species=[],[]
# for alignment in  blastp_records.alignments:
#     for hsp in alignment.hsps:
# #         print(hsp.identities)    # maybe add more 
#         if hsp.expect < E_VALUE_THRESH:
#             list_filtered_alignments.append(alignment.accession)
#             title_organism=re.search(r'\[.+\s.+\]', alignment.title)
#             if title_organism:
#                 m = re.match(r'\[.+\s.+\]', title_organism[0] )
#                 specie = m.group(0)
#                 #print(specie)
#                 list_species.append(specie)
# for x in sorted(set(list_species)):
#     print("number of times: {} that appeared specie: {}".format(list_species.count(x),x))

# print(list_filtered_alignments)

In [24]:
results_Blastp= open('blastp_file_ddx18.xml')
blastp_records = NCBIXML.read(results_Blastp)
E_VALUE_THRESH = 0.00001
list_filtered_alignments,list_species=[],[]
for alignment in  blastp_records.alignments:
    for hsp in alignment.hsps:
#         print(hsp.identities)    # maybe add more 
        if hsp.expect < E_VALUE_THRESH:
            list_filtered_alignments.append(alignment.accession)
            title_organism=re.search(r'\[.+\s.+\]', alignment.title)
            if title_organism:
                m = re.match(r'\[.+\s.+\]', title_organism[0] )
                specie = m.group(0)
                #print(specie)
                list_species.append(specie)
print('Number of times:    that appeared specie ')              
for x in sorted(set(list_species)):
        print("{:>8} - - - - {:<20}".format(list_species.count(x),x))

Number of times:    that appeared specie 
       2 - - - - [Arabidopsis thaliana]
       2 - - - - [Aspergillus clavatus NRRL 1]
       2 - - - - [Aspergillus fischeri NRRL 181]
       2 - - - - [Aspergillus fumigatus Af293]
       2 - - - - [Aspergillus nidulans FGSC A4]
       2 - - - - [Aspergillus niger CBS 513.88]
       2 - - - - [Aspergillus oryzae RIB40]
       2 - - - - [Aspergillus terreus NIH2624]
       1 - - - - [Candida albicans SC5314]
       2 - - - - [Chaetomium globosum CBS 148.51]
       2 - - - - [Coccidioides immitis RS]
       1 - - - - [Cryptococcus neoformans var. neoformans JEC21] >sp|P0CQ85.1| RecName: Full=ATP-dependent RNA helicase HAS1 [Cryptococcus neoformans var. neoformans B-3501A]
       1 - - - - [Debaryomyces hansenii CBS767]
       2 - - - - [Dictyostelium discoideum]
       1 - - - - [Drosophila melanogaster]
       1 - - - - [Eremothecium gossypii ATCC 10895]
       1 - - - - [Fusarium graminearum PH-1]
       1 - - - - [Gallus gallus]     
       

In [106]:
# with open('blastp_files_ID_ddx18.txt', 'w') as f:
#     for line in list_filtered_alignments:
#         f.write(f"{line}\n")

In [25]:
Blast=open("blastp_file_ddx18.xml")
Blast_record=NCBIXML.parse(Blast)
for x in Blast_record:
    print(x.database)
    print(x.matrix)
    print(x.gap_penalties)
    print(len(x.alignments))

swissprot
BLOSUM62
(11, 1)
50


In [26]:
result_handle=open("blastp_file_ddx18.xml")
blast_record=NCBIXML.read(result_handle)
E_VALUE_THRESH=0.0001
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect<E_VALUE_THRESH:
            print("----Aligment----")
            print("sequence:",alignment.title)
            print("lenght:",alignment.length)
            print("e.value:",hsp.expect) 
            print(hsp.query[0:75] + "...")
            print(hsp.query[0:75] + "...")
            print(hsp.sbjct[0:75] + "...")

----Aligment----
sequence: sp|Q9NVP1.2| RecName: Full=ATP-dependent RNA helicase DDX18; AltName: Full=DEAD box protein 18; AltName: Full=Myc-regulated DEAD box protein; Short=MrDb [Homo sapiens]
lenght: 670
e.value: 0.0
MSHLPMKLLRKKIEKRNLKLRQRNLKFQGASNLTLSETQNGDVSEETMGSRKVKKSKQKPMNVGLSETQNGGMSQ...
MSHLPMKLLRKKIEKRNLKLRQRNLKFQGASNLTLSETQNGDVSEETMGSRKVKKSKQKPMNVGLSETQNGGMSQ...
MSHLPMKLLRKKIEKRNLKLRQRNLKFQGASNLTLSETQNGDVSEETMGSRKVKKSKQKPMNVGLSETQNGGMSQ...
----Aligment----
sequence: sp|Q8K363.1| RecName: Full=ATP-dependent RNA helicase DDX18; AltName: Full=DEAD box protein 18 [Mus musculus]
lenght: 660
e.value: 0.0
MSHLPMKLLRKKIEKRNLKLRQRNLKFQGASNLTLSETQNGDVSEETMGSRKVKKSKQKPMNVGLSETQNGGMSQ...
MSHLPMKLLRKKIEKRNLKLRQRNLKFQGASNLTLSETQNGDVSEETMGSRKVKKSKQKPMNVGLSETQNGGMSQ...
MSQLQMKLLRRKIEKRNAKLRQRNLKLQETSDTSLSQPQNGDVPKETGKGGKVKKALKRSVPVDSAEAQSGGMPE...
----Aligment----
sequence: sp|Q9VD51.2| RecName: Full=Probable ATP-dependent RNA helicase pitchoune [Drosophila melanogaster]
lenght: 680
e.valu

In [39]:
filtragem= []
results_Blastp= open('blastp_file_ddx18.xml')
blastp_records_r = NCBIXML.read(results_Blastp)
first=blastp_records_r.alignments[0]
len_max_seq=first.hsps[0].align_length
print(len_max_seq)
results_Blastp= open('blastp_file_ddx18.xml')
blastp_records = NCBIXML.parse(results_Blastp)
list_accession,list_identities,list_coverage_start, list_coverage_stop,list_alignLen=[],[],[],[],[]
list_organism=[]
E_VALUE_THRESH = 0.00001
for alignment in  blastp_records:
    for a in alignment.alignments:
        for b in a.hsps:
            if b.expect < E_VALUE_THRESH:
                title_organism=re.search(r'\[.+\s.+\]', a.title)
                if title_organism:
                    m = re.match(r'\[(.+\s.+)\]', title_organism[0] )
                    list_accession.append(a.accession)
                    list_coverage_start.append(b.sbjct_start)
                    list_coverage_stop.append(b.sbjct_end)
                    list_identities.append(b.identities)
                    list_alignLen.append(b.align_length)
                    list_organism.append(m.group(1))
            #print(a.accession, b.align_length, b.identities, b.expect, b.sbjct_start, b.sbjct_end)
            
#calculation of %identity                        
def identity(list_i, list_a):
    perc_list_identities=[]
    for index in range(len(list_i)):
        perc_list_identities.append(round(list_i[index]*100/list_a[index]))
    return perc_list_identities

#calculation of the %coverage - for all hsps of all accessions
def coverage(list_end,list_start):
    perc_list_coverage=[]
    for index in range(len(list_identities)):
        perc_list_coverage.append(round((1+list_end[index]-list_start[index])/len_max_seq*100))
    return perc_list_coverage

#selecting the max coverage and max identity per hsps/accession
def max_values_by_id(ids, covs, idents): 
    unique_ids = []
    max_coverage = []
    max_identity = []
    for id, cov, ident in zip(ids, covs,idents):
        if id in unique_ids:
            idx = unique_ids.index(id)
            max_coverage[idx] = max(max_coverage[idx], cov)
            max_identity[idx] = max(max_identity[idx], ident)
        else:
            unique_ids.append(id)
            max_coverage.append(cov)
            max_identity.append(ident)
    return unique_ids, max_coverage, max_identity

beta_max=max_values_by_id(list_accession, coverage(list_coverage_stop, list_coverage_start),identity(list_identities,list_alignLen))

thresold_coverage=74
thresold_identity=59
print('__Thresholds__')
print('coverage: {}%   |'.format(thresold_coverage))
print('identity: {}%   |'.format(thresold_identity))
print('e-value< {} |\n'.format(E_VALUE_THRESH))
print('{:>5}{:>14}{:>11}{:>35}'.format('ID','%coverage','%identity', 'Organism'))
count=0
filtragem= []
for id, cov, ident, specie in zip(beta_max[0], beta_max[1], beta_max[2], list_organism):
    if int(cov)>thresold_coverage and ident>thresold_identity:
        if int(cov)>100:
            print('|{:>5}|{:>9}*{:>9}{:>24}'.format(id, cov, ident, specie))
            count+=1
        else:
            print('|{:>5}|{:>9}{:>10}{:>40}'.format(id, cov, ident, specie))
        filtragem.append(f"{id} {specie}")
  
if count>0:
    print('\n* means that the subject sequence is longer than the query sequence')

670
__Thresholds__
coverage: 74%   |
identity: 59%   |
e-value< 1e-05 |

   ID     %coverage  %identity                           Organism
|Q9NVP1|      100       100                            Homo sapiens
|Q8K363|       99        86                            Mus musculus
|A4R8B5|       78        60                Pyricularia oryzae 70-15
|A2Q9T6|       75        62            Aspergillus niger CBS 513.88
|A3LNR6|       75        61       Scheffersomyces stipitis CBS 6054
|Q7S2N9|       78        60                 Neurospora crassa OR74A


In [40]:
#teste
lista_sem_repetidos= []
seen = set()
for x in filtragem:
    parts = x.split()
    organi = parts[1] + '_' + parts[2]
    #print(parts)
    #print(organi)
    
    if organi in seen:
         filtragem.remove(x) 
    else:
        seen.add(organi)
        lista_sem_repetidos.append(x)      

lista_ids = []
with open('protein_result_blastp.txt', 'w') as f:
    for x in lista_sem_repetidos:
        print(x)
        IDS = x.split()
        lista_ids.append(IDS[0])
        f.write(f"{IDS[0]}\n")
    
# print(lista_ids)

Q9NVP1 Homo sapiens
Q8K363 Mus musculus
A4R8B5 Pyricularia oryzae 70-15
A2Q9T6 Aspergillus niger CBS 513.88
A3LNR6 Scheffersomyces stipitis CBS 6054
Q7S2N9 Neurospora crassa OR74A


In [111]:
# blast_qresult=SearchIO.read("blastp_file_ddx18.xml","blast-xml")
# blast_hsp=blast_qresult[0][0]
# print(blast_qresult[0][0])#best hsp
# print(blast_hsp.aln)

## Uniprot search of Blastp results

In [41]:
WEBSITE_API="https://rest.uniprot.org"
def get_url(url,**kwargs):
    response=requests.get(url,**kwargs);
    if not response.ok:
        print(response.text)
        response.raise_for_status()
        sys.exit()
    return response
r=get_url(f"{WEBSITE_API}/uniprotkb/search?query=ddx18 AND (taxonomy_id:9606) AND (reviewed:true)")#more detail possible
data=r.json()
n_results=len(data["results"])
print(f"Number of results: {n_results}\n")
total=r.headers.get("x-total-results")
page_total=len(data["results"])

Number of results: 2



In [113]:
# #one way to list the ids
# r=get_url(f"{WEBSITE_API}//uniprotkb/search?query=DDX18 AND (taxonomy_id:9606) AND (reviewed:true)&format=list&size=2")
# #r.text

# accessions=[r.text.replace("\n",",")]
# accessions

In [114]:
# #to format fasta that alow see the protein sequences and id in a form more familiary
# r=get_url(f"{WEBSITE_API}//uniprotkb/search?query=DDX18  AND (taxonomy_id:9606) AND (reviewed:true)&format=fasta")
# fasta=r.text
# print(fasta)

-we choose the first one the id Q9NVP1 

In [115]:
# with ExPASy.get_sprot_raw("Q9NVP1") as handle:
#     seq_record = SeqIO.read(handle, "swiss")
#     #print(seq_record.id)
#     #print(seq_record.entry_name, "\n")
#     #print(", ".join(seq_record.accessions), "\n")
#     #print(seq_record.keywords, "\n")
#     #print(seq_record.organism, "\n")
#     #print(len(seq_record.sequence), "aa", "\n")
#     print(seq_record.seq)
# #is not possible in my case!!!!!!!!!!!!!!!!!!!!1

We can't understand why the id of ddx18 doesn't work and the other genes already work

In [42]:
# print(lista_sem_repetidos)
a=", ".join(lista_sem_repetidos)
b=a.replace(" ","_")
c=b.split(",")
print(c)

['Q9NVP1_Homo_sapiens', '_Q8K363_Mus_musculus', '_A4R8B5_Pyricularia_oryzae_70-15', '_A2Q9T6_Aspergillus_niger_CBS_513.88', '_A3LNR6_Scheffersomyces_stipitis_CBS_6054', '_Q7S2N9_Neurospora_crassa_OR74A']


In [43]:
fileUniprot= open("protein_result_blastp.txt", "r")
fields="sequence"
WEBSITE_API="https://rest.uniprot.org"
seqs=[]

for i in fileUniprot:
    r=get_url("{}/uniprotkb/search?query={} AND (reviewed:true)&fields={}&size=1&format=tsv".format(WEBSITE_API,i, fields))
    seqs.append(str(r.content))

with open('ddx18_blastp_file.fasta', 'w') as f:
    for index, seq in enumerate(seqs):
        existe = re.search(r'b\'Sequence\\n(.+?(?=\\n\'))', seq, re.DOTALL)
        if existe:
            m = re.match( r'b\'Sequence\\n(.+?(?=\\n\'))', seq, re.DOTALL )
            print(lista_sem_repetidos[index])
            f.write(f">{c[index]}\n{m.group(1)}\n\n")                    
            print(m.group(1))


Q9NVP1 Homo sapiens
MSHLPMKLLRKKIEKRNLKLRQRNLKFQGASNLTLSETQNGDVSEETMGSRKVKKSKQKPMNVGLSETQNGGMSQEAVGNIKVTKSPQKSTVLTNGEAAMQSSNSESKKKKKKKRKMVNDAEPDTKKAKTENKGKSEEESAETTKETENNVEKPDNDEDESEVPSLPLGLTGAFEDTSFASLCNLVNENTLKAIKEMGFTNMTEIQHKSIRPLLEGRDLLAAAKTGSGKTLAFLIPAVELIVKLRFMPRNGTGVLILSPTRELAMQTFGVLKELMTHHVHTYGLIMGGSNRSAEAQKLGNGINIIVATPGRLLDHMQNTPGFMYKNLQCLVIDEADRILDVGFEEELKQIIKLLPTRRQTMLFSATQTRKVEDLARISLKKEPLYVGVDDDKANATVDGLEQGYVVCPSEKRFLLLFTFLKKNRKKKLMVFFSSCMSVKYHYELLNYIDLPVLAIHGKQKQNKRTTTFFQFCNADSGTLLCTDVAARGLDIPEVDWIVQYDPPDDPKEYIHRVGRTARGLNGRGHALLILRPEELGFLRYLKQSKVPLSEFDFSWSKISDIQSQLEKLIEKNYFLHKSAQEAYKSYIRAYDSHSLKQIFNVNNLNLPQVALSFGFKVPPFVDLNVNSNEGKQKKRGGGGGFGYQKTKKVEKSKIFKHISKKSSDSRQFSH
Q8K363 Mus musculus
MSQLQMKLLRRKIEKRNAKLRQRNLKLQETSDTSLSQPQNGDVPKETGKGGKVKKALKRSVPVDSAEAQSGGMPEETLENGKVKKSPQKLTTLANGEAAPTPPPDSEVKKKKKKKRKMANDAGPDTKKAKTEESAEACEEPEDDVKKADDSEVPSLPLGLTGAFEDTSFASLSNLVNENTLKAIEEMGFKRMTEIQHKSIRPLLEGRDLLAAAKTGSGKTLAFLIPVIELIVKLKFMPRNGTGVLILSPTRELAMQTFGVLKELMTHHVHTYGLIMGGSNRSAEVQKLL

In [44]:
fileUniprot= open("protein_result_blastp.txt", "r")
fields=['accession','organism_name','protein_name','cc_function','cc_subcellular_location']
WEBSITE_API="https://rest.uniprot.org"

def get_field_for_id(ID_PROT, field): 
    response = get_url("{}/uniprotkb/search?query={}&fields={}&size=1&format=tsv".format(WEBSITE_API,ID_PROT,field))
    return str(response.content)

def get_info_uniprot(fileUniprot):
    results = []
    result = []
    tmp = []
    uniprot_final_list=[]
    
    for first_index in fileUniprot:
        tmp= []
        for field in fields:
            result = get_field_for_id(first_index, field)
            tmp.append(result)
        results.append(tmp)
    for ind in results:
        uniprot_list=[]
        for i in ind:
            entry = re.search(r'b\'Entry\\n(.+?(?=\\n\'))', str(i), re.DOTALL)
            organism = re.search(r'b\'Organism\\n(.+?(?=\\n\'))', str(i), re.DOTALL)
            protein = re.search(r'Protein names\\n(.+?(?=\\n))', str(i), re.DOTALL)
            function = re.search(r'Function \[CC\]\\n.{9} (.+?(?=\\n))', str(i), re.DOTALL )
            location_exist = re.search(r'b\'Subcellular location \[CC\]\\nSUBCELLULAR LOCATION: (.+?(?=\\n\'))', str(i), re.DOTALL )
            location_notexist = re.search( r'b\'Subcellular location \[CC\]\\n\\n\'',str(i), re.DOTALL )   
            if entry:
                uniprot_list.append(entry.group(1))
            if organism:
                uniprot_list.append(organism.group(1))
            if protein:
                uniprot_list.append(protein.group(1))
            if location_exist:
                uniprot_list.append(location_exist.group(1))
            if location_notexist:
                uniprot_list.append('N/A')
            if function:
                uniprot_list.append(function.group(1))
        uniprot_final_list.append(uniprot_list)
    return uniprot_final_list


In [45]:
get_info_uniprot(fileUniprot)

[['Q9NVP1',
  'Homo sapiens (Human)',
  'ATP-dependent RNA helicase DDX18 (EC 3.6.4.13) (DEAD box protein 18) (Myc-regulated DEAD box protein) (MrDb)',
  'Probable RNA-dependent helicase.',
  'Nucleus, nucleolus {ECO:0000269|PubMed:16963496}. Chromosome {ECO:0000269|PubMed:20813266}.'],
 ['Q8K363',
  'Mus musculus (Mouse)',
  'ATP-dependent RNA helicase DDX18 (EC 3.6.4.13) (DEAD box protein 18)',
  'Probable RNA-dependent helicase.',
  'Nucleus, nucleolus {ECO:0000250|UniProtKB:Q9NVP1}. Chromosome {ECO:0000250|UniProtKB:Q9NVP1}.'],
 ['A4R8B5',
  'Magnaporthe oryzae (strain 70-15 / ATCC MYA-4617 / FGSC 8958) (Rice blast fungus) (Pyricularia oryzae)',
  'ATP-dependent RNA helicase HAS1 (EC 3.6.4.13)',
  'ATP-dependent RNA helicase involved in 40S ribosomal subunit biogenesis. Required for the processing and cleavage of 35S pre-rRNA at sites A0, A1, and A2, leading to mature 18S rRNA.',
  'Nucleus, nucleolus {ECO:0000250}.'],
 ['A2Q9T6',
  'Aspergillus niger (strain CBS 513.88 / FGSC A151

In [46]:
fileUniprot= open("protein_result_blastp.txt", "r")
allInfo=get_info_uniprot(fileUniprot)
for index in allInfo:
    print('Id: {}\nOrganism: {}\nProtein name: {}\nSubcelular location: {}\nFunction:  {}\n\n'.format(index[0], index[1], index[2], index[4], index[3]))
    

Id: Q9NVP1
Organism: Homo sapiens (Human)
Protein name: ATP-dependent RNA helicase DDX18 (EC 3.6.4.13) (DEAD box protein 18) (Myc-regulated DEAD box protein) (MrDb)
Subcelular location: Nucleus, nucleolus {ECO:0000269|PubMed:16963496}. Chromosome {ECO:0000269|PubMed:20813266}.
Function:  Probable RNA-dependent helicase.


Id: Q8K363
Organism: Mus musculus (Mouse)
Protein name: ATP-dependent RNA helicase DDX18 (EC 3.6.4.13) (DEAD box protein 18)
Subcelular location: Nucleus, nucleolus {ECO:0000250|UniProtKB:Q9NVP1}. Chromosome {ECO:0000250|UniProtKB:Q9NVP1}.
Function:  Probable RNA-dependent helicase.


Id: A4R8B5
Organism: Magnaporthe oryzae (strain 70-15 / ATCC MYA-4617 / FGSC 8958) (Rice blast fungus) (Pyricularia oryzae)
Protein name: ATP-dependent RNA helicase HAS1 (EC 3.6.4.13)
Subcelular location: Nucleus, nucleolus {ECO:0000250}.
Function:  ATP-dependent RNA helicase involved in 40S ribosomal subunit biogenesis. Required for the processing and cleavage of 35S pre-rRNA at sites A

In [47]:
# results_Blastp= open('blastp_file_ddx18.xml')
# blastp_records = NCBIXML.read(results_Blastp)
# titles_list=[]
# for alignment in  blastp_records.alignments:   
#     titles_list.append(alignment.title)

# file= open("protein_result_blastp.txt", "r")
# fields="sequence"
# WEBSITE_API="https://rest.uniprot.org"
# seqs=[]

# for i in file:
#     r=get_url("{}/uniprotkb/search?query={} AND (reviewed:true)&fields={}&size=1&format=tsv".format(WEBSITE_API,i, fields))
#     seqs.append(str(r.content))

# with open('ddx18_blastp_file.fasta', 'w') as f:
#     for index, seq in enumerate(seqs):
#         existe = re.search(r'b\'Sequence\\n(.+?(?=\\n\'))', seq, re.DOTALL)
#         if existe:
#             m = re.match( r'b\'Sequence\\n(.+?(?=\\n\'))', seq, re.DOTALL )
#             f.write(f">{titles_list[index]}\n{m.group(1)}\n\n")                    
# #             print(m.group(1))      

In [48]:
# file= open("blastp_files_ID_ddx18.txt", "r")
# fields="accession,organism_name,protein_name,cc_subcellular_location,cc_function"
# WEBSITE_API="https://rest.uniprot.org"
# with open('uniprot_result_CDS_filtered.txt', 'w',encoding='utf-8') as f:
#     for i in file:
#         r=get_url("{}/uniprotkb/search?query={} AND (reviewed:true)&fields={}&size=1&format=tsv".format(WEBSITE_API,i, fields))
#         print(r.text)
#         f.write(r.text)
#         f.write('\n')

# Alignment and Phylo

In [49]:

dir =r'C:\Program files (x86)\ClustalW2\clustalw2'
in_file =r'ddx18_blastp_file.fasta'

clustalw_cline = ClustalwCommandline(dir, infile=in_file)
clustalw_cline()
print(clustalw_cline)


"C:\Program files (x86)\ClustalW2\clustalw2" -infile=ddx18_blastp_file.fasta


In [50]:
cline = ClustalwCommandline("clustalw", infile="ddx18_blastp_file.fasta", outfile="ddx18_blastp_file.aln")
cline

ClustalwCommandline(cmd='clustalw', infile='ddx18_blastp_file.fasta', outfile='ddx18_blastp_file.aln')

In [51]:

align = AlignIO.read("ddx18_blastp_file.aln", "clustal")

print(format(align, "clustal")) #formato string

CLUSTAL 2.1 multiple sequence alignment


_A4R8B5_Pyricularia_oryzae_70-      -------MDDRSSKKRKLKDVNGAKASGAATDVTV-------KPKKLKKA
_Q7S2N9_Neurospora_crassa_OR74      -------MASEFSKKRKLKDAKIATEDGAATDK---------KTKKVKKD
_A2Q9T6_Aspergillus_niger_CBS_      ---MPIPVDTAKSINKKRKRKHGARAATDADDASPKPAVENGETTEVTET
_A3LNR6_Scheffersomyces_stipit      ---------MAKNVNSKANGKKSGSVKRALKEK------------KQPEP
Q9NVP1_Homo_sapiens                 MSHLPMKLLRKKIEKRNLKLRQRNLKFQGASNLTLSETQNGDVSEETMGS
_Q8K363_Mus_musculus                MSQLQMKLLRRKIEKRNAKLRQRNLKLQETSDTSLSQPQNGDVPKETGKG
                                                  : : :  :         :             :    

_A4R8B5_Pyricularia_oryzae_70-      QSQEEVEP--------KVEKGSKKEEEE-----NDWSDEEENEAADDAGH
_Q7S2N9_Neurospora_crassa_OR74      KKEKKAAE--------EVVEDATPEEE------NDAQKAEEQQDGDEVIP
_A2Q9T6_Aspergillus_niger_CBS_      PTEKKSKK--------KAAKESEKEEKPKKRKVDDSSNEDENDSEEEVRE
_A3LNR6_Scheffersomyces_stipit      EFSDSEDD--------EVDQEKLVEEL------DEDFDEVANMLGADVTD


In [52]:

align = AlignIO.read("ddx18_blastp_file.aln", "clustal")
print(align)
print("Número de linhas: %i" % len(align)) 
print(format(align, "clustal"))

for record in align:
    print("%s - %s" % (record.seq, record.id)) 

Alignment with 6 rows and 673 columns
-------MDDRSSKKRKLKDVNGAKASGAATDVTV-------KP...--- _A4R8B5_Pyricularia_oryzae_70-
-------MASEFSKKRKLKDAKIATEDGAATDK---------KT...--- _Q7S2N9_Neurospora_crassa_OR74
---MPIPVDTAKSINKKRKRKHGARAATDADDASPKPAVENGET...--- _A2Q9T6_Aspergillus_niger_CBS_
---------MAKNVNSKANGKKSGSVKRALKEK-----------...--- _A3LNR6_Scheffersomyces_stipit
MSHLPMKLLRKKIEKRNLKLRQRNLKFQGASNLTLSETQNGDVS...FSH Q9NVP1_Homo_sapiens
MSQLQMKLLRRKIEKRNAKLRQRNLKLQETSDTSLSQPQNGDVP...FSH _Q8K363_Mus_musculus
Número de linhas: 6
CLUSTAL 2.1 multiple sequence alignment


_A4R8B5_Pyricularia_oryzae_70-      -------MDDRSSKKRKLKDVNGAKASGAATDVTV-------KPKKLKKA
_Q7S2N9_Neurospora_crassa_OR74      -------MASEFSKKRKLKDAKIATEDGAATDK---------KTKKVKKD
_A2Q9T6_Aspergillus_niger_CBS_      ---MPIPVDTAKSINKKRKRKHGARAATDADDASPKPAVENGETTEVTET
_A3LNR6_Scheffersomyces_stipit      ---------MAKNVNSKANGKKSGSVKRALKEK------------KQPEP
Q9NVP1_Homo_sapiens                 MSHLPMKLLRKKIEKRNLKLRQRNLKFQGASNLTLSETQNGDVSE

In [53]:
count = AlignIO.convert("ddx18_blastp_file.aln", "clustal","ddx18_blastp_file.sth", "stockholm") 
print ("Convertidos %i alinhamentos" % count )

Convertidos 1 alinhamentos


In [54]:
alignment = AlignIO.read("ddx18_blastp_file.sth", "stockholm")

In [55]:
calculator = DistanceCalculator('blosum62')
dm = calculator.get_distance(alignment)
print(dm)

_A4R8B5_Pyricularia_oryzae_70-	0
_Q7S2N9_Neurospora_crassa_OR74	0.2306907111262334	0
_A2Q9T6_Aspergillus_niger_CBS_	0.2835671342685371	0.2753132407720962	0
_A3LNR6_Scheffersomyces_stipit	0.3435433344935608	0.3419309864064134	0.3245309242529535	0
Q9NVP1_Homo_sapiens	0.4452968645763843	0.4453125	0.46394075981970384	0.4397506925207756	0
_Q8K363_Mus_musculus	0.4512803458596608	0.44786729857819907	0.46329684481648425	0.4400690846286701	0.12711111111111106	0
	_A4R8B5_Pyricularia_oryzae_70-	_Q7S2N9_Neurospora_crassa_OR74	_A2Q9T6_Aspergillus_niger_CBS_	_A3LNR6_Scheffersomyces_stipit	Q9NVP1_Homo_sapiens	_Q8K363_Mus_musculus


In [56]:
constructor = DistanceTreeConstructor() 
upgmatree = constructor.upgma(dm)
print(upgmatree)

Tree(rooted=True)
    Clade(branch_length=0, name='Inner5')
        Clade(name='Inner1')
            Clade(name='_Q8K363_Mus_musculus')
            Clade(name='Q9NVP1_Homo_sapiens')
        Clade(name='Inner4')
            Clade(name='_A3LNR6_Scheffersomyces_stipit')
            Clade(name='Inner3')
                Clade(name='_A2Q9T6_Aspergillus_niger_CBS_')
                Clade(name='Inner2')
                    Clade(name='_Q7S2N9_Neurospora_crassa_OR74')
                    Clade(name='_A4R8B5_Pyricularia_oryzae_70-')


In [57]:
njtree = constructor.nj(dm)
print(njtree)

Tree(rooted=False)
    Clade(branch_length=0, name='Inner4')
        Clade(name='_A4R8B5_Pyricularia_oryzae_70-')
        Clade(name='_Q7S2N9_Neurospora_crassa_OR74')
        Clade(name='Inner3')
            Clade(name='Inner2')
                Clade(name='Inner1')
                    Clade(name='_Q8K363_Mus_musculus')
                    Clade(name='Q9NVP1_Homo_sapiens')
                Clade(name='_A3LNR6_Scheffersomyces_stipit')
            Clade(name='_A2Q9T6_Aspergillus_niger_CBS_')


In [58]:
Phylo.write([upgmatree, njtree],"phylotree_DDX18.nhx","newick")

2

In [59]:
Phylo.draw_ascii(njtree)

  ______________ _A4R8B5_Pyricularia_oryzae_70-
 |
 |______________ _Q7S2N9_Neurospora_crassa_OR74
 |
_|                                     ________ _Q8K363_Mus_musculus
 |        ____________________________|
 |   ____|                            |_______ Q9NVP1_Homo_sapiens
 |  |    |
 |__|    |_____________________ _A3LNR6_Scheffersomyces_stipit
    |
    |__________________ _A2Q9T6_Aspergillus_niger_CBS_



In [61]:
Phylo.draw_ascii(upgmatree)

                                  _____________ _Q8K363_Mus_musculus
  _______________________________|
 |                               |_____________ Q9NVP1_Homo_sapiens
_|
 |           __________________________________ _A3LNR6_Scheffersomyces_stipit
 |__________|
            |      ____________________________ _A2Q9T6_Aspergillus_niger_CBS_
            |_____|
                  |     _______________________ _Q7S2N9_Neurospora_crassa_OR74
                  |____|
                       |_______________________ _A4R8B5_Pyricularia_oryzae_70-

